In [ ]:
import copy
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

image_path = '../input/kermany2018/OCT2017 /'
input_size = 224

mean = [0.224, 0.224, 0.224]
std = [0.1551, 0.1551, 0.1551]

dataset_ratio = 0.9

batch_size = 128
num_epoch = 20
CLASS_NAMES = list(os.listdir(image_path + 'train'))

pretrained_model = 'inceptionv3'
train_md = 'mod'

In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(input_size),
    transforms.Normalize((mean), (std))
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(input_size),
    transforms.Normalize((mean), (std))
])

image_dataset = {
        'train': torchvision.datasets.ImageFolder(os.path.join(image_path, 'train'), train_transform),
        'val': torchvision.datasets.ImageFolder(os.path.join(image_path, 'val'), val_transform)
    }

data_loader = {
    x: torch.utils.data.DataLoader(image_dataset[x], batch_size=batch_size,
                                    shuffle=True, num_workers=4)
    for x in ['train', 'val']
}

In [ ]:
def train_model():
    accuracy_epoch = { 'train': [], 'val': [] }
    loss_epoch = { 'train': [], 'val': [] }
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epoch):

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            loop = tqdm(enumerate(data_loader[phase]), total=len(data_loader[phase]))
            loop.set_description(f"Epoch [{epoch + 1}/{num_epoch}]")

            running_loss = 0.0
            running_corrects = 0
            running_num = 0

            for batch_idx, (inputs, labels) in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_num += len(labels.data)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(labels == preds)

                loop.set_postfix(loss=(running_loss / running_num),
                                 acc=(running_corrects.double() / running_num).item())

            accuracy_epoch[phase].append((running_corrects.double() / running_num).item())
            loss_epoch[phase].append(running_loss / running_num)

            if phase == 'val' and accuracy_epoch[phase][epoch] >= best_acc:
                best_acc = accuracy_epoch[phase][epoch]
                best_model_wts = copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model_wts)

    return { 'model': model, 'accuracy': accuracy_epoch, 'loss': loss_epoch }

In [ ]:
def build_model(model_n, mmode=''):
    if model_n == 'resnet18':
        model = torchvision.models.resnet18(pretrained=True)
    elif model_n == 'inceptionv3':
        model = torchvision.models.inception_v3(pretrained=True)
        model.aux_logits = False
    elif model_n == 'resnext50_32x4d':
        model = torchvision.models.resnext50_32x4d(pretrained=True)
    else:
        return 'No model with the name'

    if mmode == '':
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 4)
    elif mmode == 'mod':
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(
            nn.BatchNorm1d(num_ftrs),
            nn.Linear(num_ftrs, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 4)
        )

    model = model.to(device)

    return model

In [ ]:
model = build_model(pretrained_model, train_md)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

output = train_model()

In [ ]:
torch.save(output, '/kaggle/working/inceptionv3-1_1.pth')